In [1]:
import pandas as pd
from openai import OpenAI
import pickle
import time
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('dodiom_it_test.csv')

In [3]:
df

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,abbaiare alla luna,Abbaiare alla luna o roteare gatti non può cur...,idiom,1.000000,2,0,0,"[0, 1, 2]","['Abbaiare', 'alla', 'luna']","['abbaiare', 'alla', 'luna', 'o', 'roteare', '...","['Abbaiare', 'alla', 'luna', 'o', 'roteare', '..."
1,abbaiare alla luna,Non è possibile che tu debba abbaiare per tutt...,idiom,0.500000,1,1,0,"[6, 9, 10]","['abbaiare', 'alla', 'luna']","['non', 'essere', 'possibile', 'che', 'tu', 'd...","['Non', 'è', 'possibile', 'che', 'tu', 'debba'..."
2,abbaiare alla luna,Chiedere alla prof di pubblicare le slide è co...,idiom,0.500000,1,1,0,"[9, 10, 11]","['abbaiare', 'alla', 'luna']","['chiedere', 'alla', 'prof', 'di', 'pubblicare...","['Chiedere', 'alla', 'prof', 'di', 'pubblicare..."
3,abbaiare alla luna,Abbai almeno 10 volte al giorno alla luna,idiom,0.000000,0,2,0,"[0, 6, 7]","['Abbai', 'alla', 'luna']","['abbaiare', 'almeno', '10', 'volta', 'al', 'g...","['Abbai', 'almeno', '10', 'volte', 'al', 'gior..."
4,abbaiare alla luna,Sprechi troppo tempo ad abbaiare alla luna,idiom,0.666667,2,1,0,"[4, 5, 6]","['abbaiare', 'alla', 'luna']","['sprecare', 'troppo', 'tempo', 'ad', 'abbaiar...","['Sprechi', 'troppo', 'tempo', 'ad', 'abbaiare..."
...,...,...,...,...,...,...,...,...,...,...,...
2279,voltare la faccia,"Puoi voltare la faccia per favore, voglio fart...",nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['potere', 'voltare', 'la', 'faccia', 'per', '...","['Puoi', 'voltare', 'la', 'faccia', 'per', 'fa..."
2280,voltare la faccia,per favore puoi voltare la faccia verso di me ...,nonidiom,1.000000,3,0,0,"[3, 5]","['voltare', 'faccia']","['per', 'favore', 'potere', 'voltare', 'la', '...","['per', 'favore', 'puoi', 'voltare', 'la', 'fa..."
2281,voltare la faccia,devi voltare la faccia da entrambi i lati per ...,nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['deviare', 'voltare', 'la', 'faccia', 'da', '...","['devi', 'voltare', 'la', 'faccia', 'da', 'ent..."
2282,voltare la faccia,Appena mi ha visto ha voltato la faccia,nonidiom,0.500000,1,1,0,"[5, 7]","['voltato', 'faccia']","['appena', 'mi', 'avere', 'vistare', 'avere', ...","['Appena', 'mi', 'ha', 'visto', 'ha', 'voltato..."


In [22]:
idioms = df["submission"].tolist()

In [23]:
client = OpenAI(api_key="OpenAI KEY")

In [24]:
idioms

['Abbaiare alla luna o roteare gatti non può curare una milza malata.',
 'Non è possibile che tu debba abbaiare per tutto alla luna',
 'Chiedere alla prof di pubblicare le slide è come abbaiare alla luna',
 'Abbai almeno 10 volte al giorno alla luna',
 'Sprechi troppo tempo ad abbaiare alla luna',
 'Trump si lamenta dei risultati delle ultime elezioni, ma sta abbaiando alla luna.',
 'parlare con te è come abbaiare alla luna',
 'Trump si lamenta dei risultati delle ultime elezioni, ma sta abbaiando alla luna.',
 'Trump si lamenta dei risultati delle ultime elezioni, ma sta abbaiando alla luna.',
 'Non mi piace abbaiare alla luna',
 'se dodiom non mi mostra più la classifica non mi resta che abbaiare all’infinito alla luna',
 'so che è inutile ma non posso fare a meno di abbaiare alla luna in questa situazione',
 "Ho abbaiato alla luna quando l'ho saputo",
 'Devi abbaiare costantemente alla luna?',
 'inutile abbaiare alla luna se non ti ascolta',
 'Non voglio abbaiare più alla luna',
 'A

In [25]:
results = []
for t in tqdm(range(len(idioms))):  # Loop over the range of idioms
    i = idioms[t]  # Access each idiom directly using the loop index
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
    {"role": "system", "content": 
        "Il tuo compito è quello di essere un modello linguistico che parla italiano. Rispondi seguendo esclusivamente le istruzioni fornite e non scrivere nient'altro."
    },
    {"role": "user", "content": 
        "Voglio che tu identifichi se la frase qui sotto contiene o meno un'espressione idiomatica. Se la frase contiene un'espressione idiomatica, scrivi solo '1'. Se non contiene, scrivi solo '0'. Fornisci solo '0' o '1' come risposta, e nient'altro. Frase:\n" + i
    }
]

    )
    results.append(completion)  # Append the result to the list


  0%|          | 0/2284 [00:00<?, ?it/s]

In [26]:
results

[ChatCompletion(id='chatcmpl-AFlsJD6yS3SVZQ7bcN3GlSRkXcbD6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1728322315, model='gpt-4-turbo-2024-04-09', object='chat.completion', service_tier=None, system_fingerprint='fp_68a5bb159e', usage=CompletionUsage(completion_tokens=1, prompt_tokens=154, total_tokens=155, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))),
 ChatCompletion(id='chatcmpl-AFlsKogWHaHQTlxeaZgoy5TWd5pCj', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1728322316, model='gpt-4-turbo-2024-04-09', object='chat.completion', service_tier=None, system_fingerprint='fp_68a5bb159e', usage=C

In [27]:
with open('gpt-4-turbo_1_it.pkl', 'wb') as f:
      pickle.dump(results, f)

In [4]:
labels = df["category"].tolist()

In [29]:
df = pd.DataFrame([i.choices[0].message.content for i in results])
df.to_csv('results_gpt-4-turbo_it.csv', index=False)

In [30]:
df

,0
0,1
1,1
2,1
3,1
4,1
...,...
2279,0
2280,0
2281,0
2282,1


In [31]:
for i in [i for i, x in enumerate(df[0].tolist()) if x == "1\n"]:
    df[0][i] = "1"

In [32]:
df

,0
0,1
1,1
2,1
3,1
4,1
...,...
2279,0
2280,0
2281,0
2282,1


In [6]:
gpt_labels = [int(i) for i in df.iloc[:, 0].tolist()]


In [7]:
human_labels = [1 if i == "idiom" else 0 for i in labels]

In [35]:
gpt_labels

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,


In [36]:
human_labels

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,


In [37]:
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(human_labels, gpt_labels)

print(report)


              precision    recall  f1-score   support

           0       0.74      0.57      0.64       939
           1       0.74      0.86      0.80      1345

    accuracy                           0.74      2284
   macro avg       0.74      0.71      0.72      2284
weighted avg       0.74      0.74      0.73      2284



In [8]:
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(human_labels, gpt_labels, digits=4)

print(report)


              precision    recall  f1-score   support

           0     0.7379    0.5666    0.6410       939
           1     0.7396    0.8595    0.7950      1345

    accuracy                         0.7391      2284
   macro avg     0.7387    0.7130    0.7180      2284
weighted avg     0.7389    0.7391    0.7317      2284

